In [20]:
import numpy as np
import pandas as pd
from scipy.fft import fft, ifft
import cmath

from processing_func import from_S_to_T, from_T_to_S
from file_opener import run

In [21]:
%matplotlib widget
import matplotlib.pyplot as plt

Импорт данных

In [29]:
list_of_angles = [0, 30, 45, 60]

In [30]:
for angle in list_of_angles:
    tempTE = pd.read_csv('ND_struct_' + str(angle) + 'deg_s11_TE.txt', sep='\t', header=None, skiprows=3, 
                names=['Frequency', 'ReSz11', 'ImSz11'])
    tempTM = pd.read_csv('ND_struct_' + str(angle) + 'deg_s11_TM.txt', sep='\t', header=None, skiprows=3, 
                names=['Frequency', 'ReSz11', 'ImSz11'])

    if angle == 0:
        tempTE['Angle'] = angle
        tempTM['Angle'] = angle
        dataTE = tempTE.loc[tempTE['Frequency'] == 25]
        dataTM = tempTM.loc[tempTM['Frequency'] == 25]
    else:
        tempTE['Angle'] = angle
        tempTM['Angle'] = angle
        dataTE = pd.concat([dataTE, tempTE.loc[tempTE['Frequency'] == 25]], ignore_index=True)
        dataTM = pd.concat([dataTM, tempTM.loc[tempTE['Frequency'] == 25]], ignore_index=True)

In [31]:
dataTE.head()

,Frequency,ReSz11,ImSz11,Angle
0,25.0,0.904027,0.050718,0
1,25.0,0.808577,-0.338870,30
2,25.0,0.833809,0.182458,45
3,25.0,0.748441,-0.184718,60


In [32]:
dataTM.head()

,Frequency,ReSz11,ImSz11,Angle
0,25.0,0.904027,0.050718,0
1,25.0,0.719588,-0.523267,30
2,25.0,0.720663,-0.509632,45
3,25.0,-0.312453,-0.855562,60


In [26]:
reflecting_coef_30deg_TE = pd.read_csv('ND_struct_30deg_s11_TE.txt', sep='\t', header=None, skiprows=3, 
                names=['Frequency', 'ReSz11', 'ImSz11'])

reflecting_coef_0deg_TE = pd.read_csv('ND_struct_0deg_s11_TE.txt', sep='\t', header=None, skiprows=3, 
                names=['Frequency', 'ReSz11', 'ImSz11'])

reflecting_coef_45deg_TE = pd.read_csv('ND_struct_45deg_s11_TE.txt', sep='\t', header=None, skiprows=3, 
                names=['Frequency', 'ReSz11', 'ImSz11'])

reflecting_coef_60deg_TE = pd.read_csv('ND_struct_60deg_s11_TE.txt', sep='\t', header=None, skiprows=3, 
                names=['Frequency', 'ReSz11', 'ImSz11'])

In [27]:
mes_list = [reflecting_coef_0deg_TE, reflecting_coef_30deg_TE, reflecting_coef_45deg_TE, reflecting_coef_60deg_TE]

data_25GHz = mes_list[0].loc[mes_list[0]['Frequency'] == 25]

for i in range(1, len(mes_list), 1):
    data_25GHz = pd.concat([data_25GHz, mes_list[i].loc[mes_list[i]['Frequency'] == 25]], ignore_index=True)

In [35]:
data_25GHz

,Frequency,ReSz11,ImSz11
0,25.0,0.904027,0.050718
1,25.0,0.808577,-0.338870
2,25.0,0.833809,0.182458
3,25.0,0.748441,-0.184718


In [34]:
dataTE

,Frequency,ReSz11,ImSz11,Angle
0,25.0,0.904027,0.050718,0
1,25.0,0.808577,-0.338870,30
2,25.0,0.833809,0.182458,45
3,25.0,0.748441,-0.184718,60
